In [1]:
#Author: Muhammad Azizi Bin Mohd Ariffin
#Email: mazizi@fskm.uitm.edu.my
#Description: Data pre-processing for deep learning traffic classification

import pyshark
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [208]:
inputPath = "D:\\dataset\\ISCX-VPN2016\\CompletePCAPs\\"
outputPath = "D:\\dataset\\ISCX-VPN2016\\extract2\\"
fileName = "skype_chat1a"
fileExtension = ".pcap"
label = 'skypechat'

In [209]:
#c = pyshark.FileCapture("D:\\dataset\\ISCX-VPN2016\\CompletePCAPs\\aim_chat_3a.pcap", use_json=True, include_raw=True)
input = inputPath + fileName + fileExtension
c = pyshark.FileCapture(input, use_json=True, include_raw=True)
print("Done Read PCAP")

Done Read PCAP


In [210]:
#c[637]

In [211]:
#print(dir(c[637]))

In [212]:
#checkString = str(c[637].layers)
#print(checkString)
#print(type(checkString))
#if 'RTCPs' in checkString :
#    print('ok')

In [213]:
countDf = 0
df = pd.DataFrame()
while countDf <= 1479 :
    index = 'B' + str(countDf)
    df[index] = np.nan
    countDf += 1

#print(df)

def padding(byteValue) :
    byteValue = byteValue.ljust(2960, '0')
    return byteValue

def truncate(byteValue) :
    byteValue = byteValue[0:2960]
    return byteValue

def processPacket(byteValue) :
    byteLen = len(byteValue)
    if byteLen > 2960 :
        byteValue = truncate(byteValue)
    elif byteLen < 2960 :
        byteValue = padding(byteValue)
    #print(byteValue)
    #print(len(byteValue))
    n = 2
    byteList = []
    for i in range(0, len(byteValue), n):
        temp = byteValue[i:i+n]
        base16INT = int(temp, 16)
        byteList.append(base16INT)
    #x_array = np.array(byteList)
    #normalized_arr = preprocessing.normalize([x_array])
    #normalized_arr = normalized_arr.flatten()
    #return normalized_arr
    #replace append, numpy, pandas
    return byteList

count = 0
countIndex = 0
n = 2
counttls = 0
countdns = 0
counttcp = 0
countudp = 0
counthttp = 0
countmdns = 0
countdtls = 0
countstun = 0
countgquic = 0
countrtcp = 0
checkString = ""

for x in c :
    print(count)
    if 'TLS' in c[count] :
        #print('TLS')
        counttls += 1
        try:
            byteValue = c[count].ip_raw.value + c[count].tcp_raw.value + c[count].tls_raw.value
        except:
            byteValue = c[count].ip_raw.value + c[count].tcp_raw.value + c[count].tls_raw.value[0]
        #print(byteValue)
        #print(len(byteValue))
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
        #break
    elif 'HTTP' in c[count] :
        counthttp += 1
        try : 
            byteValue = c[count].ip_raw.value + c[count].tcp_raw.value + c[count].http_raw.value + c[count].data_raw.value
        except :
            byteValue = c[count].ip_raw.value + c[count].tcp_raw.value + c[count].http_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'MDNS' in c[count] :
        countmdns += 1
        try :
            byteValue = c[count].ipv6_raw.value + c[count].udp_raw.value + c[count].mdns_raw.value
        except :
            byteValue = c[count].ip_raw.value + c[count].udp_raw.value + c[count].mdns_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'DNS' in c[count] and 'UDP' in c[count] :
        #print('DNS')
        countdns += 1
        byteValue = c[count].ip_raw.value + c[count].udp_raw.value + c[count].dns_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'DTLS' in c[count] :
        countdtls +=1
        c[count].ip_raw.value + c[count].udp_raw.value + c[count].dtls_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'GQUIC' in c[count] :
        countgquic += 1
        c[count].ip_raw.value + c[count].udp_raw.value + c[count].gquic_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'RTCP' in c[count] :
        countrtcp += 1
        c[count].ip_raw.value + c[count].udp_raw.value + c[count].rtcp_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'STUN' in c[count] :
        countstun += 1
        try :
            c[count].ip_raw.value + c[count].udp_raw.value + c[count].stun_raw.value
        except :
            try :
                c[count].ip_raw.value + c[count].tcp_raw.value + c[count].stun_raw.value
            except :
                c[count].ip_raw.value + c[count].stun_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'TCP' in c[count] :
        #print('TCP')
        if 'DATA' in c[count] :
            counthttp += 1
            try :
                byteValue = c[count].ip_raw.value + c[count].tcp_raw.value + c[count].http_raw.value + c[count].data_raw.value
                counthttp += 1
            except :
                byteValue = c[count].ip_raw.value + c[count].tcp_raw.value + c[count].data_raw.value
                counttcp += 1
        else :
            counttcp += 1
            byteValue = c[count].ip_raw.value + c[count].tcp_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'UDP' in c[count] and 'DATA' in c[count] :
        #print('UDP')
        countudp += 1
        byteValue = c[count].ip_raw.value + c[count].udp_raw.value + c[count].data_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    else :
        checkString = str(c[count].layers)
        if 'RTCP' in checkString :
            c[count].ip_raw.value + c[count].udp_raw.value + c[count].rtcp_raw.value
            df.loc[countIndex] = processPacket(byteValue)
            countIndex += 1
            countrtcp += 1
            print('RTCP')
        else :
            print(c[count].highest_layer)  
    count += 1
df['label'] = label
#print(df)
print('Done')

0
ARP_RAW
1
NBNS_RAW
2
NBNS_RAW
3
LLMNR_RAW
4
LLMNR_RAW
5
LLMNR_RAW
6
LLMNR_RAW
7
8
NBNS_RAW
9
LLMNR_RAW
10
LLMNR_RAW
11
LLMNR_RAW
12
LLMNR_RAW
13
NBNS_RAW
14
LLMNR_RAW
15
LLMNR_RAW
16
ARP_RAW
17
NBNS_RAW
18
LLMNR_RAW
19
LLMNR_RAW
20
LLMNR_RAW
21
LLMNR_RAW
22
NBNS_RAW
23
ARP_RAW
24
ARP_RAW
25
LLMNR_RAW
26
LLMNR_RAW
27
LLMNR_RAW
28
LLMNR_RAW
29
LLMNR_RAW
30
LLMNR_RAW
31
LLMNR_RAW
32
LLMNR_RAW
33
LLMNR_RAW
34
LLMNR_RAW
35
ARP_RAW
36
37
38
39
40
41
42
43
44
45
46
LLMNR_RAW
47
LLMNR_RAW
48
LLMNR_RAW
49
LLMNR_RAW
50
ARP_RAW
51
NBNS_RAW
52
53
54
55
56
NBNS_RAW
57
NBNS_RAW
58
ARP_RAW
59
ARP_RAW
60
NBNS_RAW
61
NBNS_RAW
62
ARP_RAW
63
ARP_RAW
64
ARP_RAW
65
DHCPV6_RAW
66
NBNS_RAW
67
NBNS_RAW
68
NBNS_RAW
69
NBNS_RAW
70
ARP_RAW
71
72
73
74
75
76
SSDP_RAW
77
78
79
80
81
ARP_RAW
82
DB-LSP-DISC_RAW
83
DB-LSP-DISC_RAW
84
NBNS_RAW
85
NBNS_RAW
86
ARP_RAW
87
88
ARP_RAW
89
90
ARP_RAW
91
NBNS_RAW
92
ARP_RAW
93
ARP_RAW
94
ARP_RAW
95
NBNS_RAW
96
NBNS_RAW
97
LLMNR_RAW
98
LLMNR_RAW
99
ARP_RAW
100
NBNS_RAW
101
L

718
719
ARP_RAW
720
DHCPV6_RAW
721
722
ARP_RAW
723
ARP_RAW
724
725
ARP_RAW
726
ARP_RAW
727
728
NBNS_RAW
729
SSDP_RAW
730
DHCPV6_RAW
731
ARP_RAW
732
NBNS_RAW
733
LLMNR_RAW
734
LLMNR_RAW
735
LLMNR_RAW
736
LLMNR_RAW
737
ARP_RAW
738
DHCPV6_RAW
739
DHCPV6_RAW
740
ARP_RAW
741
ARP_RAW
742
743
NBNS_RAW
744
ARP_RAW
745
746
LLMNR_RAW
747
LLMNR_RAW
748
LLMNR_RAW
749
LLMNR_RAW
750
ARP_RAW
751
ARP_RAW
752
NBNS_RAW
753
NBNS_RAW
754
ARP_RAW
755
ARP_RAW
756
ARP_RAW
757
ARP_RAW
758
ARP_RAW
759
ARP_RAW
760
NBNS_RAW
761
NBNS_RAW
762
ARP_RAW
763
ARP_RAW
764
STP_RAW
765
ARP_RAW
766
NBNS_RAW
767
LLMNR_RAW
768
LLMNR_RAW
769
770
771
772
773
774
775
NBNS_RAW
776
777
778
779
780
781
782
783
ARP_RAW
784
ARP_RAW
785
ARP_RAW
786
ARP_RAW
787
ARP_RAW
788
ARP_RAW
789
ARP_RAW
790
ARP_RAW
791
ARP_RAW
792
ARP_RAW
793
ARP_RAW
794
ARP_RAW
795
ARP_RAW
796
ARP_RAW
797
ARP_RAW
798
ARP_RAW
799
ARP_RAW
800
ARP_RAW
801
ARP_RAW
802
ARP_RAW
803
ARP_RAW
804
ARP_RAW
805
ARP_RAW
806
ARP_RAW
807
ARP_RAW
808
ARP_RAW
809
ARP_RAW
810
AR

1391
ARP_RAW
1392
ARP_RAW
1393
ARP_RAW
1394
ARP_RAW
1395
ARP_RAW
1396
ARP_RAW
1397
ARP_RAW
1398
ARP_RAW
1399
ARP_RAW
1400
ARP_RAW
1401
ARP_RAW
1402
ARP_RAW
1403
NBNS_RAW
1404
ARP_RAW
1405
ARP_RAW
1406
ARP_RAW
1407
ARP_RAW
1408
ARP_RAW
1409
ARP_RAW
1410
ARP_RAW
1411
ARP_RAW
1412
ARP_RAW
1413
ARP_RAW
1414
ARP_RAW
1415
ARP_RAW
1416
ARP_RAW
1417
ARP_RAW
1418
ARP_RAW
1419
ARP_RAW
1420
ARP_RAW
1421
ARP_RAW
1422
ARP_RAW
1423
ARP_RAW
1424
ARP_RAW
1425
ARP_RAW
1426
ARP_RAW
1427
ARP_RAW
1428
ARP_RAW
1429
NBNS_RAW
1430
ARP_RAW
1431
ARP_RAW
1432
NBNS_RAW
1433
ARP_RAW
1434
ARP_RAW
1435
NBNS_RAW
1436
1437
1438
IGMP_RAW
1439
ICMPV6_RAW
1440
IGMP_RAW
1441
IGMP_RAW
1442
IGMP_RAW
1443
ICMPV6_RAW
1444
ICMPV6_RAW
1445
IGMP_RAW
1446
ICMPV6_RAW
1447
ARP_RAW
1448
NBNS_RAW
1449
ICMPV6_RAW
1450
ICMPV6_RAW
1451
NBNS_RAW
1452
ICMPV6_RAW
1453
IGMP_RAW
1454
ICMPV6_RAW
1455
ICMPV6_RAW
1456
NBNS_RAW
1457
ICMPV6_RAW
1458
NBNS_RAW
1459
ICMPV6_RAW
1460
ARP_RAW
1461
ICMPV6_RAW
1462
ARP_RAW
1463
ICMPV6_RAW
1464
ARP_RAW
1

2134
2135
2136
ARP_RAW
2137
2138
NBNS_RAW
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
ARP_RAW
2149
2150
2151
2152
ARP_RAW
2153
2154
2155
2156
2157
2158
2159
ARP_RAW
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
ARP_RAW
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
NBNS_RAW
2199
DHCPV6_RAW
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
NBNS_RAW
2210
LLMNR_RAW
2211
LLMNR_RAW
2212
LLMNR_RAW
2213
LLMNR_RAW
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2274
2275
2276
2277
2278
ARP_RAW
2279
2280
2281
2282
2283
2284
2285
STP_RAW
2286
2287
2288
LLMNR_RAW
2289
LLMNR_RAW
2290
LLMNR_RAW
2291
LLMNR_RAW
2292
2293
2294
2295
2296
2297
2298
2

3031
NBNS_RAW
3032
NBNS_RAW
3033
LLMNR_RAW
3034
LLMNR_RAW
3035
NBNS_RAW
3036
LLMNR_RAW
3037
LLMNR_RAW
3038
ARP_RAW
3039
NBNS_RAW
3040
STP_RAW
3041
LLMNR_RAW
3042
LLMNR_RAW
3043
NBNS_RAW
3044
LLMNR_RAW
3045
LLMNR_RAW
3046
LLMNR_RAW
3047
LLMNR_RAW
3048
LLMNR_RAW
3049
LLMNR_RAW
3050
ARP_RAW
3051
NBNS_RAW
3052
NBNS_RAW
3053
NBNS_RAW
3054
NBNS_RAW
3055
NBNS_RAW
3056
NBNS_RAW
3057
LLMNR_RAW
3058
LLMNR_RAW
3059
LLMNR_RAW
3060
LLMNR_RAW
3061
NBNS_RAW
3062
ARP_RAW
3063
LLMNR_RAW
3064
LLMNR_RAW
3065
LLMNR_RAW
3066
LLMNR_RAW
3067
ARP_RAW
3068
3069
NBNS_RAW
3070
3071
DHCPV6_RAW
3072
ARP_RAW
3073
NBNS_RAW
3074
LLMNR_RAW
3075
LLMNR_RAW
3076
LLMNR_RAW
3077
LLMNR_RAW
3078
ARP_RAW
3079
ARP_RAW
3080
NBNS_RAW
3081
ARP_RAW
3082
NBNS_RAW
3083
ARP_RAW
3084
ARP_RAW
3085
NBNS_RAW
3086
NBNS_RAW
3087
LLMNR_RAW
3088
LLMNR_RAW
3089
NBNS_RAW
3090
NBNS_RAW
3091
3092
3093
3094
3095
ARP_RAW
3096
ARP_RAW
3097
ARP_RAW
3098
LLMNR_RAW
3099
LLMNR_RAW
3100
NBNS_RAW
3101
ARP_RAW
3102
ARP_RAW
3103
ARP_RAW
3104
ARP_RAW
3105
A

...

47775
47776
47777
47778
47779
47780
ARP_RAW
47781
ARP_RAW
47782
NBNS_RAW
47783
ARP_RAW
47784
NBNS_RAW
47785
47786
DHCPV6_RAW
47787
47788
47789
47790
ARP_RAW
47791
47792
47793
NBNS_RAW
47794
ARP_RAW
47795
NBNS_RAW
47796
ARP_RAW
47797
ARP_RAW
47798
DHCPV6_RAW
47799
47800
47801
47802
47803
47804
47805
47806
47807
47808
47809
47810
47811
47812
47813
47814
47815
47816
47817
47818
47819
ARP_RAW
47820
47821
47822
47823
47824
47825
47826
47827
47828
47829
47830
47831
47832
47833
47834
47835
47836
47837
47838
47839
47840
47841
47842
47843
47844
47845
47846
47847
47848
47849
47850
47851
47852
47853
NBNS_RAW
47854
47855
STP_RAW
47856
ARP_RAW
47857
NBNS_RAW
47858
ARP_RAW
47859
ARP_RAW
47860
47861
47862
NBNS_RAW
47863
47864
NBNS_RAW
47865
LLMNR_RAW
47866
LLMNR_RAW
47867
LLMNR_RAW
47868
LLMNR_RAW
47869
47870
NBNS_RAW
47871
ARP_RAW
47872
LLMNR_RAW
47873
LLMNR_RAW
47874
LLMNR_RAW
47875
LLMNR_RAW
47876
NBNS_RAW
47877
ARP_RAW
47878
ARP_RAW
47879
47880
47881
NBNS_RAW
47882
DB-LSP-DISC_RAW
47883
DB-LSP-DI

48668
48669
48670
48671
48672
48673
48674
48675
48676
48677
48678
48679
48680
48681
48682
48683
48684
48685
48686
48687
48688
48689
SNMP_RAW
48690
SNMP_RAW
48691
48692
48693
48694
48695
48696
48697
48698
48699
48700
48701
48702
48703
48704
48705
48706
48707
48708
48709
48710
48711
SNMP_RAW
48712
48713
48714
48715
48716
48717
48718
48719
48720
48721
48722
DUPLICATE IP ADDRESS DETECTED FOR 192.168.0.120 (78:2B:CB:20:90:90) - ALSO IN USE BY 84:2B:2B:4B:82:9C (FRAME 47539)
48723
48724
48725
48726
ARP_RAW
48727
48728
48729
48730
48731
48732
NBNS_RAW
48733
48734
NBNS_RAW
48735
ARP_RAW
48736
ARP_RAW
48737
48738
48739
48740
NBNS_RAW
48741
NBNS_RAW
48742
ARP_RAW
48743
ARP_RAW
48744
NBNS_RAW
48745
DB-LSP-DISC_RAW
48746
DB-LSP-DISC_RAW
48747
DB-LSP-DISC_RAW
48748
STP_RAW
48749
48750
48751
48752
48753
48754
48755
48756
48757
48758
48759
48760
48761
48762
48763
48764
48765
48766
48767
48768
48769
48770
48771
48772
48773
48774
48775
48776
48777
48778
48779
48780
48781
48782
48783
48784
48785
48786
4

49513
ARP_RAW
49514
NBNS_RAW
49515
LLMNR_RAW
49516
LLMNR_RAW
49517
LLMNR_RAW
49518
LLMNR_RAW
49519
49520
ARP_RAW
49521
LLMNR_RAW
49522
LLMNR_RAW
49523
LLMNR_RAW
49524
LLMNR_RAW
49525
49526
49527
49528
49529
49530
49531
49532
49533
49534
49535
49536
49537
49538
49539
49540
49541
49542
49543
49544
49545
49546
49547
49548
49549
49550
NBNS_RAW
49551
ARP_RAW
49552
NBNS_RAW
49553
49554
49555
49556
49557
49558
49559
49560
49561
49562
49563
49564
NBNS_RAW
49565
LLMNR_RAW
49566
LLMNR_RAW
49567
LLMNR_RAW
49568
LLMNR_RAW
49569
LLMNR_RAW
49570
LLMNR_RAW
49571
LLMNR_RAW
49572
LLMNR_RAW
49573
NBNS_RAW
49574
NBNS_RAW
49575
NBNS_RAW
49576
LLMNR_RAW
49577
LLMNR_RAW
49578
LLMNR_RAW
49579
LLMNR_RAW
49580
49581
49582
49583
NBNS_RAW
49584
DB-LSP-DISC_RAW
49585
DB-LSP-DISC_RAW
49586
LLMNR_RAW
49587
LLMNR_RAW
49588
LLMNR_RAW
49589
LLMNR_RAW
49590
ARP_RAW
49591
NBNS_RAW
49592
LLMNR_RAW
49593
LLMNR_RAW
49594
LLMNR_RAW
49595
LLMNR_RAW
49596
NBNS_RAW
49597
ARP_RAW
49598
ARP_RAW
49599
ARP_RAW
49600
ARP_RAW
49601


50277
50278
NBNS_RAW
50279
50280
50281
ARP_RAW
50282
50283
50284
50285
50286
50287
50288
50289
50290
50291
50292
50293
50294
50295
NBNS_RAW
50296
LLMNR_RAW
50297
LLMNR_RAW
50298
LLMNR_RAW
50299
LLMNR_RAW
50300
50301
ARP_RAW
50302
50303
NTP_RAW
50304
NBNS_RAW
50305
LLMNR_RAW
50306
LLMNR_RAW
50307
LLMNR_RAW
50308
LLMNR_RAW
50309
STP_RAW
50310
LLMNR_RAW
50311
LLMNR_RAW
50312
LLMNR_RAW
50313
LLMNR_RAW
50314
LLMNR_RAW
50315
LLMNR_RAW
50316
LLMNR_RAW
50317
LLMNR_RAW
50318
50319
50320
NBNS_RAW
50321
NBNS_RAW
50322
50323
50324
50325
50326
50327
50328
50329
50330
50331
50332
ARP_RAW
50333
ARP_RAW
50334
50335
ARP_RAW
50336
50337
50338
50339
50340
50341
NBNS_RAW
50342
LLMNR_RAW
50343
LLMNR_RAW
50344
ARP_RAW
50345
NBNS_RAW
50346
50347
BROWSER_RAW
50348
NBNS_RAW
50349
ARP_RAW
50350
50351
LLMNR_RAW
50352
LLMNR_RAW
50353
ARP_RAW
50354
ARP_RAW
50355
ARP_RAW
50356
ARP_RAW
50357
ARP_RAW
50358
ARP_RAW
50359
ARP_RAW
50360
ARP_RAW
50361
ARP_RAW
50362
ARP_RAW
50363
ARP_RAW
50364
ARP_RAW
50365
ARP_RAW
50366


AttributeError: No attribute named ip_raw

In [204]:
#c[792]

In [205]:
#print(dir(c[792]))

In [206]:
output = outputPath + fileName + fileExtension + '.csv'
df.to_csv(output, index=False)
print(output)

D:\dataset\ISCX-VPN2016\extract2\ICQchat2.pcapng.csv


In [207]:
print('Total packets inspected:', count)
print('Total packets extracted:', counttls + countdns + counttcp + countudp + counthttp + countmdns + countdtls + countstun + countgquic + countrtcp)
print('Total TLS:', counttls)
print('Total DNS:', countdns)
print('Total TCP:', counttcp)
print('Total UDP:', countudp)
print('Total HTTP:', counthttp)
print('Total MDNS:', countmdns)
print('Total DTLS:', countdtls)
print('Total STUN:', countstun)
print('Total GQUIC:', countgquic)
print('Total RTCP:', countrtcp)

Total packets inspected: 280
Total packets extracted: 223
Total TLS: 103
Total DNS: 12
Total TCP: 102
Total UDP: 4
Total HTTP: 2
Total MDNS: 0
Total DTLS: 0
Total STUN: 0
Total GQUIC: 0
Total RTCP: 0
